In [1]:
import os
import math
import talib
import random
import pickle
import json
import pandas as pd
import numpy as np
import jqdatasdk as jq
import baostock as bs
import time

from multiprocessing import Pool
from functools import partial
from scipy import signal
from scipy import stats
from datetime import datetime, timedelta
from tqdm import tqdm

import matplotlib.pyplot as plt
from IPython.core.display import HTML, clear_output

%matplotlib inline
%config InlineBackend.figure_format='svg'
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',    5000)

from featureutil2 import FeatureTool
from plotutils import PlotTool
from datagent import BaosAgent
from datagent import DataUtil
from datagent import JQDataUtil
from sfstrategy import SFStrategy

def get_freq_file_sto_ids(freq):
    file_sto_ids = []
    path_src = './data/' + str(freq) + '/'
    files = os.listdir(path_src)
    all_file_sto_id = []
    for file in files:
        if file.endswith('pkl'):
            file_sto_ids.append(file[:6])

    return file_sto_ids

### 备选条件：过去3个月中连续2周的k_01d大于80

In [ ]:
delta = 15
daily_count = int(240/delta)
factor_01d = daily_count

def find_alternatives(sub_sto_ids):
    sub_alternative_sto_ids = {}
    for sto_id in tqdm(sub_sto_ids):
        tmp_df = BaosAgent.read_freq_df(delta, sto_id)
        tmp_df['k_01d'], tmp_df['d_01d'] = talib.STOCH(tmp_df.high, tmp_df.low, tmp_df.close, fastk_period=9*factor_01d, slowk_period=3*factor_01d, slowd_period=3*factor_01d)
        tmp_df = tmp_df.dropna()
        tmp_df = tmp_df.reset_index(drop=True)
        if len(tmp_df) < daily_count * 5 * 4 * 6:
            continue
#         ### 取最近120天的数据集
#         tmp_df = tmp_df.iloc[-daily_count * 5 * 4 * 6:]
#         tmp_df = tmp_df.dropna()
#         tmp_df = tmp_df.reset_index(drop=True)
        
        cond = (tmp_df.k_01d > 80).rolling(daily_count * 5 * 2).apply(lambda x: x.all(), raw=True) == True
        if len(tmp_df[cond]) > 0:
            sub_alternative_sto_ids[sto_id] = {'ts':tmp_df[cond].time.values}
    return sub_alternative_sto_ids

def find_alternatives_callback(rlts):
    alternative_sto_infos = {}
    for rlt in rlts:
        for key, value in rlt.items():
            alternative_sto_infos[key] = value
    return alternative_sto_infos


all_sto_ids = get_freq_file_sto_ids(delta)

NUM_PROCESS = 6
size = math.ceil(len(all_sto_ids) / NUM_PROCESS)
sub_seqs = []
for proc_idx in range(NUM_PROCESS):
    start_idx = size * proc_idx
    stop_idx = size * (proc_idx + 1)
    sub_seq = all_sto_ids[start_idx:stop_idx]
    sub_seqs.append(sub_seq)

with Pool(NUM_PROCESS) as pool:
    rlts = pool.map(find_alternatives, sub_seqs)
    alternative_sto_infos = find_alternatives_callback(rlts)
    print(len(alternative_sto_infos))

### 备选条件：过去15天中有10天的k_01d大于80

In [ ]:
delta = 15
daily_count = int(240/delta)
factor_01d = daily_count

def over80_over10din20d(datas):
    if len(datas[datas > 80]) > daily_count * 10:
        return True
    return False

def find_alternatives(sub_sto_ids):
    sub_alternative_sto_ids = {}
    for sto_id in tqdm(sub_sto_ids):
        tmp_df = BaosAgent.read_freq_df(delta, sto_id)
        tmp_df['k_01d'], tmp_df['d_01d'] = talib.STOCH(tmp_df.high, tmp_df.low, tmp_df.close, fastk_period=9*factor_01d, slowk_period=3*factor_01d, slowd_period=3*factor_01d)
        tmp_df = tmp_df.dropna()
        tmp_df = tmp_df.reset_index(drop=True)
        if len(tmp_df) < daily_count * 5 * 4:
            continue

        ### 取最近X天的数据集
        tmp_df = tmp_df.iloc[-daily_count * 60:]
        tmp_df = tmp_df.dropna()
        tmp_df = tmp_df.reset_index(drop=True)
        
        cond = tmp_df.k_01d.rolling(daily_count * 15).apply(over80_over10din20d, raw=True)
        cond = cond.fillna(.0)
        cond = cond.replace(0, False)
        cond = cond.replace(1, True)
        cross_20_idxes = tmp_df[(tmp_df.k_01d > 20) & (tmp_df.k_01d.shift(1) < 20)].index
        
        if len(cross_20_idxes) == 0:
            continue

        if len(tmp_df[cond]) > 0 and cross_20_idxes[-1] > (len(tmp_df) - daily_count):
            sub_alternative_sto_ids[sto_id] = {'ts':tmp_df[cond].time.values}
    return sub_alternative_sto_ids

def find_alternatives_callback(rlts):
    alternative_sto_infos = {}
    for rlt in rlts:
        for key, value in rlt.items():
            alternative_sto_infos[key] = value
    return alternative_sto_infos


all_sto_ids = get_freq_file_sto_ids(delta)

NUM_PROCESS = 6
size = math.ceil(len(all_sto_ids) / NUM_PROCESS)
sub_seqs = []
for proc_idx in range(NUM_PROCESS):
    start_idx = size * proc_idx
    stop_idx = size * (proc_idx + 1)
    sub_seq = all_sto_ids[start_idx:stop_idx]
    sub_seqs.append(sub_seq)

with Pool(NUM_PROCESS) as pool:
    rlts = pool.map(find_alternatives, sub_seqs)
    alternative_sto_infos = find_alternatives_callback(rlts)
    print(len(alternative_sto_infos))

In [ ]:
# delta = 15
# sto_ids = get_freq_file_sto_ids(delta)
# df = BaosAgent.read_freq_df(delta, random.sample(sto_ids, 1)[0])
# df.iloc[-1].time

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()
jq.get_query_count()

factor_01d=1
daily_count=1

# sto_id = random.sample(alternative_sto_infos.keys(), 1)[0]
sto_id = '600338'
# for sto_id in alternative_sto_infos.keys():
tmp_df = BaosAgent.read_freq_df(delta, sto_id)
tmp_df = DataUtil.add_rt_data(sto_id, tmp_df, delta, JQDataUtil.get_end_ts(delta))
tmp_df['k_01d'], tmp_df['d_01d'] = talib.STOCH(tmp_df.high, tmp_df.low, tmp_df.close, fastk_period=9*factor_01d, slowk_period=3*factor_01d, slowd_period=3*factor_01d)
tmp_df = tmp_df.iloc[-daily_count * 300:]
tmp_df = tmp_df.dropna()
tmp_df = tmp_df.reset_index(drop=True)

bs.login()
bs_sto_id = BaosAgent.get_baostock_sto_id(sto_id)
sto_name = bs.query_stock_basic(code=bs_sto_id).get_row_data()[1]
print(sto_id, sto_name)
bs.logout()

fig = plt.figure(figsize=(len(tmp_df)/128, 10))
fig.patch.set_facecolor('#2A2A2A')
xaxis_idxes = tmp_df.time.str[:4] + '-' + tmp_df.time.str[4:6] + '-' + tmp_df.time.str[6:8] + ' ' + tmp_df.time.str[8:10] + ':' + tmp_df.time.str[10:12]
x_axis_slice = slice(0, -1, daily_count * 5 * 4)

############## subplot 1 ##############
ax = plt.subplot(2,1,1)
PlotTool.custom_ax(ax)
# ax.plot(xaxis_idxes, tmp_df.volume, color='cyan', lw=.5)
ax.fill_between(xaxis_idxes, tmp_df.volume, facecolor='cyan', alpha=0.3)
PlotTool.custom_axticks(ax, xaxis_idxes, x_axis_slice)
ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, tmp_df.close, color='orange', lw=.5)
PlotTool.custom_axticks(ax2, xaxis_idxes, x_axis_slice)

############## subplot 2 ##############
ax = plt.subplot(2,1,2)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, tmp_df.k_01d, lw=.2, color='cyan')
ax.hlines(80, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
ax.hlines(50, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
ax.hlines(20, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
PlotTool.custom_axticks(ax, xaxis_idxes, x_axis_slice)

ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, tmp_df.close, color='orange', lw=.5)
# target_ts = alternative_sto_infos[sto_id]['ts']
# target_idxes = tmp_df[tmp_df.time.isin(target_ts)].index
# min_y, max_y = np.min(tmp_df.close), np.max(tmp_df.close)
# for target_idx in target_idxes:
#     ax2.vlines(xaxis_idxes[target_idx], min_y, max_y, lw=.2, color='yellow')
PlotTool.custom_axticks(ax2, xaxis_idxes, x_axis_slice)

plt.show()

In [13]:
%run MF_Util.ipynb

import re
import requests

def display_sto_info(sto_id):
    bs.login()
    bs_sto_id = BaosAgent.get_baostock_sto_id(sto_id)
    sto_name = bs.query_stock_basic(code=bs_sto_id).get_row_data()[1]
    print(sto_id, sto_name)
#     bs.logout()

def get_latest_sums(mfs):
    sums = []
    for idx in range(len(mfs)):
        sub_sum = mfs.iloc[:idx+1].sum()
        sums.append(sub_sum)
    return sums

def prepare_mf_data(sto_id, days):
    dates, closes, chgrts, slarges, larges, middles, smalls, mfs = get_mf_from_sina(sto_id, days)
    arrays = np.array([dates, closes, slarges, larges, middles, smalls, mfs])
    mf_df = pd.DataFrame(data=arrays.transpose(), columns=['dates', 'closes', 'slarges', 'larges', 'middles', 'smalls', 'mfs'])
    mf_df = mf_df.astype({'closes':float, 'slarges':float, 'larges':float, 'middles':float, 'smalls':float, 'mfs':float})
    slarges_sum = get_latest_sums(mf_df.slarges)
    larges_sum = get_latest_sums(mf_df.larges)
    middles_sum = get_latest_sums(mf_df.middles)
    smalls_sum = get_latest_sums(mf_df.smalls)
    mfs_sum = get_latest_sums(mf_df.mfs)
    
    mf_df['slarges_sum'] = slarges_sum
    mf_df['larges_sum'] = larges_sum
    mf_df['middles_sum'] = middles_sum
    mf_df['smalls_sum'] = smalls_sum
    mf_df['mfs_sum'] = mfs_sum
    
    return mf_df

def plot_mf(sto_id, mf_df, save=False):

    fig = plt.figure(figsize=(36, 9))
    fig.patch.set_facecolor('#2A2A2A')
    xaxis_idxes = mf_df.dates
    x_axis_slice = slice(0, -1, 20)

    ############## subplot 1 ##############
    ax = plt.subplot(3,1,1)
    PlotTool.custom_ax(ax)
    ax.bar(xaxis_idxes, mf_df.slarges, facecolor='orangered', alpha=.5)
    ax.bar(xaxis_idxes, mf_df.larges,  facecolor='yellow',   alpha=.5)
    # ax.bar(dates, mf_df.middles, facecolor='cyan',   alpha=.5)
    # ax.fill_between(dates, mf_df.smalls , facecolor='lime', alpha=.2)

    PlotTool.custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, mf_df.closes, color='orange', lw=.5)
    # ax2.hlines(17.17, xmin=min(dates), xmax=max(dates), color='w', lw=.5)
    PlotTool.custom_axticks(ax2, xaxis_idxes, x_axis_slice)

    ############## subplot 2 ##############
    ax = plt.subplot(3,1,2)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, mf_df.slarges_sum, color='orangered', lw=.5)
    ax.plot(xaxis_idxes, mf_df.larges_sum,  color='yellow', lw=.5)
    ax.plot(xaxis_idxes, mf_df.middles_sum, color='cyan',   lw=.5)
    ax.plot(xaxis_idxes, mf_df.smalls_sum , color='lime',   lw=.5)
    ax.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.5)
    PlotTool.custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, mf_df.closes, color='orange', lw=.5)
    PlotTool.custom_axticks(ax2, xaxis_idxes, x_axis_slice)
    
    ############## subplot 3 ##############
    ax = plt.subplot(3,1,3)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, mf_df.mfs_sum, color='orangered', lw=.5)
    ax.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.5)
    PlotTool.custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, mf_df.closes, color='orange', lw=.5)
    PlotTool.custom_axticks(ax2, xaxis_idxes, x_axis_slice)

    if save:
        full_fn = './target/' + sto_id + '.png'
        plt.savefig(full_fn, bbox_inches='tight', dpi=300, transparent=True)
    else:
        plt.show()
    plt.close('all')
    
def plot_ta(tmp_df):
    fig = plt.figure(figsize=(36, 16))
    fig.patch.set_facecolor('#2A2A2A')
    xaxis_idxes = tmp_df.time.str[:4] + '-' + tmp_df.time.str[4:6] + '-' + tmp_df.time.str[6:8] + ' ' + tmp_df.time.str[8:10] + ':' + tmp_df.time.str[10:12]
    x_axis_slice = slice(0, -1, daily_count * 5 * 4)
    
    num_subplots = 4
    idx_subplot = 0
    ############## subplot ##############
    idx_subplot += 1
    ax = plt.subplot(num_subplots,1,idx_subplot)
    PlotTool.custom_ax(ax)
    ax.fill_between(xaxis_idxes, tmp_df.volume/100, facecolor='cyan', alpha=0.3)
    PlotTool.custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, tmp_df.close, color='orange', lw=.5)
    
    cond = tmp_df.k_01d > tmp_df.d_01d
    cond = tmp_df.macdhist_01d > 0
    cond = cond & (tmp_df.cci_ema2 > 0)
    ax2.fill_between(xaxis_idxes, np.min(tmp_df.close), np.max(tmp_df.close), where=cond, facecolor='orange', alpha=.3)
    
    PlotTool.custom_axticks(ax2, xaxis_idxes, x_axis_slice)

    ############## subplot ##############
    idx_subplot += 1
    ax = plt.subplot(num_subplots,1,idx_subplot)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, tmp_df.k_01d, lw=.3, color='cyan')
    ax.plot(xaxis_idxes, tmp_df.d_01d, lw=.6, color='cyan')
#     ax.plot(xaxis_idxes, tmp_df.k_05d, lw=.3, color='cyan')
#     ax.plot(xaxis_idxes, tmp_df.d_05d, lw=.6, color='cyan')
    ax.hlines(80, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
    ax.hlines(50, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
    ax.hlines(20, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
    ax.fill_between(xaxis_idxes, 50, tmp_df.k_01d, where=tmp_df.k_01d > tmp_df.d_01d, facecolor='gold', alpha=0.3)
#     ax.fill_between(xaxis_idxes, 50, tmp_df.d_01d, where=tmp_df.d_01d < 50, facecolor='lime', alpha=0.3)
#     ax.fill_between(xaxis_idxes, 50, tmp_df.d_05d, where=tmp_df.d_05d > 50, facecolor='gold', alpha=0.3)
#     ax.fill_between(xaxis_idxes, 50, tmp_df.d_05d, where=tmp_df.d_05d < 50, facecolor='lime', alpha=0.3)
    
    PlotTool.custom_axticks(ax, xaxis_idxes, x_axis_slice)

    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, tmp_df.close, color='orange', lw=.5)
    # target_ts = alternative_sto_infos[sto_id]['ts']
    # target_idxes = tmp_df[tmp_df.time.isin(target_ts)].index
    # min_y, max_y = np.min(tmp_df.close), np.max(tmp_df.close)
    # for target_idx in target_idxes:
    #     ax2.vlines(xaxis_idxes[target_idx], min_y, max_y, lw=.2, color='yellow')
    PlotTool.custom_axticks(ax2, xaxis_idxes, x_axis_slice)

    ############## subplot ##############
    idx_subplot += 1
    ax = plt.subplot(num_subplots,1,idx_subplot)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, tmp_df.macd_01d, lw=.5, color='orangered')
    ax.plot(xaxis_idxes, tmp_df.macdsignal_01d, lw=.5, color='yellow')
    ax.fill_between(xaxis_idxes, 0, tmp_df.macdhist_01d, where=tmp_df.macdhist_01d > 0, facecolor='orangered', alpha=.5)
    ax.fill_between(xaxis_idxes, 0, tmp_df.macdhist_01d, where=tmp_df.macdhist_01d < 0, facecolor='lime',      alpha=.5)
#     ax.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
    PlotTool.custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, tmp_df.close, color='orange', lw=.5)
    PlotTool.custom_axticks(ax2, xaxis_idxes, x_axis_slice)
    
    ############## subplot ##############
    idx_subplot += 1
    ax = plt.subplot(num_subplots,1,idx_subplot)
    PlotTool.custom_ax(ax)
#     ax.fill_between(xaxis_idxes, tmp_df.cci, lw=.5, color='orangered')
#     ax.plot(xaxis_idxes, tmp_df.cci_ema2, lw=.5, color='yellow')
#     ax.fill_between(xaxis_idxes, tmp_df.cci_ema2, facecolor='orangered', alpha=.5)
    ax.fill_between(xaxis_idxes, 0, tmp_df.cci_ema2, where=tmp_df.cci_ema2 > 0, facecolor='orangered', alpha=.5)
    ax.fill_between(xaxis_idxes, 0, tmp_df.cci_ema2, where=tmp_df.cci_ema2 < 0, facecolor='lime',      alpha=.5)
#     ax.plot(xaxis_idxes, tmp_df.cci_ema3, lw=.5, color='orangered')
#     ax.fill_between(xaxis_idxes, tmp_df.cci_ema3, lw=.5, facecolor='orangered', alpha=.5)
#     ax.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
    
    PlotTool.custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, tmp_df.close, color='orange', lw=.5)
    PlotTool.custom_axticks(ax2, xaxis_idxes, x_axis_slice)

    plt.show()
    plt.close('all')


In [3]:
# jq.auth('15808061188', 'allan2jq')
# clear_output()
# jq.get_query_count()

delta = 15
all_sto_ids = get_freq_file_sto_ids(delta)

In [ ]:
alternative_df = pd.read_csv('alternative_df.csv', dtype={'sto_id':str})

In [ ]:
cci = talib.CCI(tmp_df.high, tmp_df.low, tmp_df.close)

In [17]:
%%time

sto_id = random.sample(all_sto_ids, 1)[0]
# sto_id = random.sample(alternative_001_sto_ids, 1)[0]
# sto_id = '600712'
# sto_id = '300184'

display_sto_info(sto_id)

delta = 15
daily_count = int(240/delta)
tmp_df = BaosAgent.read_freq_df(delta, sto_id)
# tmp_df = DataUtil.add_rt_data(sto_id, tmp_df, delta, JQDataUtil.get_end_ts(delta))
tmp_df['k_01d'], tmp_df['d_01d'] = talib.STOCH(tmp_df.high, tmp_df.low, tmp_df.close, fastk_period=9*daily_count,   slowk_period=9*daily_count,   slowd_period=9*daily_count)
tmp_df['k_05d'], tmp_df['d_05d'] = talib.STOCH(tmp_df.high, tmp_df.low, tmp_df.close, fastk_period=9*daily_count*5, slowk_period=9*daily_count*5, slowd_period=9*daily_count*5)
tmp_df['macd_01d'], tmp_df['macdsignal_01d'], tmp_df['macdhist_01d'] = talib.MACD(tmp_df.close, fastperiod=12*daily_count, slowperiod=26*daily_count, signalperiod=9*daily_count)
tmp_df['cci'] = talib.CCI(tmp_df.high, tmp_df.low, tmp_df.close, timeperiod=daily_count*10)
tmp_df['cci_ema2'] = talib.EMA(tmp_df.cci, timeperiod=daily_count)
# tmp_df['cci_ema2'] = talib.EMA(tmp_df.cci_ema2, timeperiod=daily_count*2)
# tmp_df['cci_ema2'] = talib.EMA(tmp_df.cci_ema2, timeperiod=daily_count*2)

tmp_df['cci_ema3'] = talib.EMA(tmp_df.cci, timeperiod=daily_count*5)
# tmp_df['cci_ema3'] = talib.EMA(tmp_df.cci_ema3, timeperiod=daily_count*2)
# tmp_df['cci_ema3'] = talib.EMA(tmp_df.cci_ema3, timeperiod=daily_count*2)

tmp_df = tmp_df.dropna()
tmp_df = tmp_df[-daily_count*240*2:]
tmp_df = tmp_df.reset_index(drop=True)
plot_ta(tmp_df)

# bg_date = tmp_df.iloc[0].date
# mf_df = prepare_mf_data(sto_id, 240 * 5)
# mf_df = mf_df[mf_df.closes > 0]
# mf_df = mf_df[mf_df.dates >= bg_date]
# plot_mf(sto_id, mf_df)


002597 金禾实业


CPU times: user 4.12 s, sys: 61 ms, total: 4.18 s
Wall time: 4.31 s


In [ ]:
def find_alternatives(sub_sto_ids):
    sub_alternative_sto_ids = []
    for sto_id in tqdm(sub_sto_ids):
        tmp_df = BaosAgent.read_freq_df(delta, sto_id)
        tmp_df['macd_01d'], tmp_df['macdsignal_01d'], tmp_df['macdhist_01d'] = talib.MACD(tmp_df.close, fastperiod=15*daily_count, slowperiod=30*daily_count, signalperiod=10*daily_count)
        tmp_df = tmp_df.dropna()
        tmp_df = tmp_df.reset_index(drop=True)
        if len(tmp_df) < daily_count * 120:
            continue
        
        obs_idx = -daily_count*5
        if tmp_df.iloc[obs_idx].macdhist_01d > 0 and tmp_df.iloc[obs_idx-1].macdhist_01d < 0:# and tmp_df.iloc[obs_idx-1].macdsignal_01d > 0:
            display_sto_info(sto_id)
            sub_alternative_sto_ids.append(sto_id)
    return sub_alternative_sto_ids

def find_alternatives_callback(rlts):
    alternative_sto_ids = []
    for rlt in rlts:
        alternative_sto_ids.extend(rlt)
    return alternative_sto_ids

delta = 15
daily_count = int(240/delta)
all_sto_ids = get_freq_file_sto_ids(delta)

# alternative_sto_ids = find_alternatives(all_sto_ids)

NUM_PROCESS = 6
size = math.ceil(len(all_sto_ids) / NUM_PROCESS)
sub_seqs = []
for proc_idx in range(NUM_PROCESS):
    start_idx = size * proc_idx
    stop_idx = size * (proc_idx + 1)
    sub_seq = all_sto_ids[start_idx:stop_idx]
    sub_seqs.append(sub_seq)

with Pool(NUM_PROCESS) as pool:
    rlts = pool.map(find_alternatives, sub_seqs)
    alternative_001_sto_ids = find_alternatives_callback(rlts)
    print(len(alternative_001_sto_ids))

In [ ]:
from scipy import stats
### slarges_slope up, larges_slope up
def find_alternatives(sub_sto_ids):
    sub_alternative_sto_ids = []
    for sto_id in tqdm(sub_sto_ids):
        mf_df = prepare_mf_data(sto_id, 240*3)
        if len(mf_df) >= 50:
            slarges_sum_slope = stats.linregress(range(50), mf_df.slarges_sum.iloc[-50:]).slope
            larges_sum_slope  = stats.linregress(range(50), mf_df.larges_sum.iloc[-50:]).slope
            if slarges_sum_slope > 2 and larges_sum_slope > 2:
                display_sto_info(sto_id)
                sub_alternative_sto_ids.append(sto_id)
    return sub_alternative_sto_ids

def find_alternatives_callback(rlts):
    alternative_sto_ids = []
    for rlt in rlts:
        alternative_sto_ids.extend(rlt)
    return alternative_sto_ids

alternative_sto_ids = find_alternatives(all_sto_ids)

In [ ]:
from scipy import stats
### slarges and larges sum slope up
def find_alternatives(sub_sto_ids):
    sub_alternative_sto_ids = []
    for sto_id in tqdm(sub_sto_ids):
        mf_df = prepare_mf_data(sto_id, 240*3)
        mf_df = mf_df[mf_df.closes > 0]
        mf_df = mf_df[:-120]
        if len(mf_df) >= 30:
            mfs_sum_slope = stats.linregress(range(30), mf_df.mfs_sum.iloc[-30:]).slope
            cp = mf_df.iloc[-1].closes
            min_p = np.min(mf_df.closes)
            cr = (cp - min_p) / min_p
            if mfs_sum_slope > 100 and mf_df.iloc[-1].mfs_sum > 50000 and cr < 0.5:
                display_sto_info(sto_id)
                print("{:>.2f} {:>.2f} {}".format(mfs_sum_slope, mf_df.iloc[-1].mfs_sum, mf_df.iloc[-1].dates))
                # plot_mf(sto_id, mf_df)
                sub_alternative_sto_ids.append(sto_id)
    return sub_alternative_sto_ids

def find_alternatives_callback(rlts):
    alternative_sto_ids = []
    for rlt in rlts:
        alternative_sto_ids.extend(rlt)
    return alternative_sto_ids



In [ ]:
delta = 15
all_sto_ids = get_freq_file_sto_ids(delta)
alternative_001_sto_ids = find_alternatives(all_sto_ids)

In [ ]:
# alternative_df = pd.DataFrame(data=alternative_001_sto_ids, columns=['sto_id'])
# alternative_df = alternative_df.sort_values(by='sto_id')
# alternative_df.to_csv('alternative_df.csv', header=True, index=False)
alternative_df = pd.read_csv('alternative_df.csv', dtype={'sto_id':str})

In [ ]:
# delta = 15
# all_sto_ids = get_freq_file_sto_ids(delta)

# NUM_PROCESS = 2
# size = math.ceil(len(all_sto_ids) / NUM_PROCESS)
# sub_seqs = []
# for proc_idx in range(NUM_PROCESS):
#     start_idx = size * proc_idx
#     stop_idx = size * (proc_idx + 1)
#     sub_seq = all_sto_ids[start_idx:stop_idx]
#     sub_seqs.append(sub_seq)

# with Pool(NUM_PROCESS) as pool:
#     rlts = pool.map(find_alternatives, sub_seqs)
#     alternative_001_sto_ids = find_alternatives_callback(rlts)
#     print(len(alternative_001_sto_ids))